## Set random seed to get more reproducible result

In [1]:
from numpy.random import seed
seed(2)
from tensorflow.random import set_seed
set_seed(2)

## Import libraries and functions

In [3]:
import os
import numpy as np
import pandas as pd

from expert.src.utils import read_genus_abu, read_labels, load_otlg, zero_weight_unk, parse_otlg, get_dmax
from expert.src.preprocessing import *
from expert.src.model import *
from expert.CLI.CLI_utils import find_pkg_resource as find_expert_resource

from tensorflow.keras.layers import Dense, Dropout, AlphaDropout
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC, BinaryAccuracy
from tensorflow.keras.initializers import HeUniform, GlorotUniform

## Construct ontology using mapper file of source samples

In [11]:
!awk -F ',' '{print $4}' exp_0/SourceMapper_0.csv | grep -v "Env" | sort | uniq  > microbiomes.txt
!expert construct -i microbiomes.txt -o ontology.pkl

Reading microbiome structure...
Generating Ontology...
100%|██████████████████████████████████████████| 3/3 [00:00<00:00, 13706.88it/s]
root
├── root:adenoma
├── root:carcinoma
└── root:normal

Done


## Set hyper-parameters for training process

In [4]:
init = HeUniform(seed=2)
sig_init = GlorotUniform(seed=2)
phylogeny_path = find_expert_resource('resources/phylogeny.csv')
ontology = load_otlg('ontology.pkl')
phylogeny = pd.read_csv(phylogeny_path, index_col=0)
lrreducer = ReduceLROnPlateau(monitor='val_loss', patience=5, min_lr=1e-5, verbose=5, factor=0.1)
stopper = EarlyStopping(monitor='val_loss', patience=15, verbose=5, restore_best_weights=True)
callbacks = [lrreducer, stopper]
phylogeny = pd.read_csv(find_expert_resource('resources/phylogeny.csv'), index_col=0)
optimizer = Adam(lr=1e-4)
metrics = [BinaryAccuracy(name='acc'), AUC(name='AUC')]
loss = BinaryCrossentropy()
epochs = 2000
batch_size = 32
validation_split = 0.1

## Preprocess the data, using EXPERT's command-line API 

In [21]:
%%bash -s "$phylogeny_path" "$phylogeny"

for((i=0; i<5; i++)); do \
ls exp_$i/QueryCM_$i.tsv > tmp; expert convert -i tmp --in-cm -o exp_$i/QueryCM_$i.h5; \
ls exp_$i/SourceCM_$i.tsv > tmp; expert convert -i tmp --in-cm -o exp_$i/SourceCM_$i.h5; \
expert map --to-otlg -t ontology.pkl -i exp_$i/SourceMapper_$i.csv -o exp_$i/SourceLabels_$i.h5; \
expert map --to-otlg -t ontology.pkl -i exp_$i/QueryMapper_$i.csv -o exp_$i/QueryLabels_$i.h5; \
done
rm tmp

running...
Reading and concatenating data, this could be slow if you have huge amount of data
db file: /home/chonghui/.etetoolkit/taxa.sqlite
        SRR2761368    SRR2761286  ...   SRR2761073   SRR2761072
count   864.000000    864.000000  ...   864.000000   864.000000
mean     13.164352     64.079861  ...    13.928241    10.555556
std     156.377417   1572.456088  ...   210.361702    79.561511
min       0.000000      0.000000  ...     0.000000     0.000000
50%       0.000000      0.000000  ...     0.000000     0.000000
max    2737.000000  46002.000000  ...  5340.000000  1188.000000

[6 rows x 17 columns]
Initializing in-memory taxonomy database for ultra-fast querying.
db file: /home/chonghui/.etetoolkit/taxa.sqlite
There will be 0/864 entries droped cause they are not in NCBI taxanomy database
Series([], dtype: object)
Extracting lineages for taxonomic entries, this may take a few minutes
Filling samples in phylogeny matrix
        SRR2761368    SRR2761286  ...   SRR2761073   SRR2761

100%|██████████| 2/2 [00:00<00:00, 40.85it/s]


## Customize and train the model 

In [10]:
class TinyModel(Model):
    
	def init_base_block(self, num_features):
		block = tf.keras.Sequential(name='base')
		block.add(Flatten()) # (1000, )
		block.add(Dropout(0.5, seed=2))
		block.add(Dense(2**6, kernel_initializer=init))
		block.add(Activation('relu')) # (1024, )
		block.add(Dense(2**5, kernel_initializer=init))
		block.add(Activation('relu')) # (1024, )
		return block
    
	def init_inter_block(self, index, name, n_units):
		k = index
		block = tf.keras.Sequential(name=name)
		block.add(Dropout(0.7, seed=2))
		return block
    
	def _init_integ_block(self, index, name, n_units):
		block = tf.keras.Sequential(name=name)
		k = index
		block.add(Dense(self._get_n_units(2**6), name='l' + str(k) + '_integ_fc0', kernel_initializer=sig_init))
		block.add(Activation('relu'))
		return block
    
    
exp=0
X, idx = read_genus_abu('exp_{0}/SourceCM_{0}.h5'.format(exp))
Y = read_labels('exp_{0}/SourceLabels_{0}.h5'.format(exp), shuffle_idx=idx, dmax=get_dmax('exp_{0}/SourceLabels_{0}.h5'.format(exp)))
IDs = sorted(list(set(X.index.to_list()).intersection(Y[0].index.to_list())))
X = X.loc[IDs, :]
Y = [y.loc[IDs, :] for y in Y]
print('Total matched samples:', sum(X.index == Y[0].index))

model = TinyModel(phylogeny=phylogeny, num_features=X.shape[1],  ontology=ontology)

# Feature encoding and standardization
X = model.encoder.predict(X, batch_size=batch_size)
X = X.reshape(X.shape[0], X.shape[1] * X.shape[2])
print('N. NaN in input features:', np.isnan(X).sum())
model.update_statistics(mean=X.mean(axis=0), std=X.std(axis=0))
X = model.standardize(X)

# Sample weight "zero" to mask unknown samples' contribution to loss
sample_weight = [zero_weight_unk(y=y, sample_weight=np.ones(y.shape[0])) for i, y in enumerate(Y)]
Y = [y.drop(columns=['Unknown']) for y in Y]

model.nn.compile(optimizer=optimizer, loss=loss, weighted_metrics=metrics)
model.nn.fit(X, Y, validation_split=validation_split, batch_size=batch_size, epochs=epochs, sample_weight=sample_weight, callbacks=callbacks)

Total matched samples: 153
N. NaN in input features: 0
           mean       std
0      0.000000  0.000000
1      0.000000  0.000000
2      0.000000  0.000000
3      0.000000  0.000000
4      0.000000  0.000000
...         ...       ...
18013  0.000580  0.002977
18014  0.000000  0.000000
18015  0.001645  0.007644
18016  0.000479  0.003096
18017  0.000000  0.000000

[18018 rows x 2 columns]
Epoch 1/2000
5/5 [==============================] - 0s 95ms/step - loss: 0.7711 - acc: 0.5033 - AUC: 0.4650 - val_loss: 0.6665 - val_acc: 0.6042 - val_AUC: 0.5693
Epoch 2/2000
5/5 [==============================] - 0s 8ms/step - loss: 0.7300 - acc: 0.5669 - AUC: 0.5088 - val_loss: 0.6661 - val_acc: 0.5625 - val_AUC: 0.5674
Epoch 3/2000
5/5 [==============================] - 0s 7ms/step - loss: 0.7449 - acc: 0.5304 - AUC: 0.5001 - val_loss: 0.6734 - val_acc: 0.6250 - val_AUC: 0.5312
Epoch 4/2000
5/5 [==============================] - 0s 7ms/step - loss: 0.6998 - acc: 0.5645 - AUC: 0.5495 - val_loss: 0

In [104]:
model.nn.summary()

Model: "functional_69"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_35 (InputLayer)        [(None, 18018)]           0         
_________________________________________________________________
base (Sequential)            (None, 512)               18976256  
_________________________________________________________________
l2_inter (Sequential)        (None, 512)               0         
_________________________________________________________________
l2_integration (Sequential)  (None, 32)                16416     
_________________________________________________________________
l2o (Sequential)             (None, 3)                 99        
Total params: 18,992,771
Trainable params: 18,992,771
Non-trainable params: 0
_________________________________________________________________
